In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

data_dir = './data'

import cv2
import numpy as np
import time
from datetime import datetime
import os
import random as rand
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plot
#from utils.data_utils import *
from utils.vis_utils import *
from utils.layer_utils import *
from utils.print_utils import *
from utils.resnet import *
import pickle
import problem_unittests as tests
import helper
from glob import glob

In [ ]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
#import problem_unittests as tests
import tarfile

cifar10_dataset_folder_path = 'cifar-10-batches-py'

# Use Floyd's cifar-10 dataset if present
floyd_cifar10_location = '/input/cifar-10/python.tar.gz'
if isfile(floyd_cifar10_location):
    tar_gz_path = floyd_cifar10_location
else:
    tar_gz_path = 'cifar-10-python.tar.gz'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(tar_gz_path):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='CIFAR-10 Dataset') as pbar:
        urlretrieve(
            'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz',
            tar_gz_path,
            pbar.hook)

if not isdir(cifar10_dataset_folder_path):
    with tarfile.open(tar_gz_path) as tar:
        tar.extractall()
        tar.close()


#tests.test_folder_path(cifar10_dataset_folder_path)

In [ ]:
batch_id = 1
sample_id = 5
helper.display_stats(cifar10_dataset_folder_path, batch_id, sample_id)

In [2]:
##Ready for the Data
def _process_and_save(normalize, features, labels, filename):
    """
    Preprocess data and save it to file
    """
    features = normalize(features)
    labels = labels
    f = open(filename, 'wb')
    pickle.dump((features, labels), f)
    f.close()

def preprocess_and_data(loadFiles, normalize):
    """
    Preprocess Training and Validation Data
    """
    n_batches = 5
    valid_features = []
    valid_labels = []
    features, labels, test_feature, test_labels, val_features, val_labels = loadFiles(True, sample_rate = 0.5)
    
    _process_and_save(normalize, features, labels, 'preprocess_batch_tranning.p')
    _process_and_save(normalize, test_feature, test_labels, 'preprocess_batch_testing.p')
    _process_and_save(normalize, val_features, val_labels, 'preprocess_batch_val.p')

    

#cap2 = cv2.VideoCapture('Pig_Identification_Qualification_Train/train/1.mp4')
def center_crop(img):
    shape = img.shape
    h = shape[0]
    w = shape[1]
    c = shape[2]
    if w > h:
        lbound = int((w - h) / 2)
        rbound = w - lbound
        return img[:, lbound:rbound, :]
    else:
        tbound = int((h - w) / 2)
        bbound = h - tbound
        return img[tbound:bbound, :, :]


def LoadVideo(path):
    cap = cv2.VideoCapture(path)
    nbFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    images = []

    for i in range(nbFrames):
        ret, frame = cap.read()
        #print(ret)
        if ret:
            images.append(center_crop(frame[:,:,::-1])) #convert data from BGR color space to RGB
    cap.release()
    cv2.destroyAllWindows()
    return images, nbFrames
    

#imgs, frameCount = LoadVideo('Pig_Identification_Qualification_Train/train/1.mp4')
#TestImg  = imgs[rand.randint(0, frameCount)]
#pImg = Image.fromarray(TestImg, mode='RGB')
#pImg.show()

#while(1):
#    ret, frame = cap.read()
#    cv2.imshow('pigtest', frame)
#    if cv2.waitKey(100) & 0xff == ord('q'):
#        break

In [3]:

def CreateDataSet(video, labels, rate=0.8):
    video = np.array(video)
    print('video Shape:{}'.format(video.shape))
    ids = np.arange(video.shape[0])
    rand.shuffle(ids)
    trainingCount = int(len(video) * rate)
    testCount = len(video)-trainingCount
    trainingVideo = []
    testVideo = []
    trainingLabels = []
    testLabels = []
    #print(len(video))
    #print(range(trainingCount))
    #print(range(testCount))
    #print(video[0].shape)
    for i in range(trainingCount-1) :
        trainingVideo.append(video[ids[i]])
        trainingLabels.append(labels[ids[i]])
    for j in range(testCount-1):
        testVideo.append(video[ids[trainingCount+j]])
        testLabels.append(labels[ids[trainingCount+j]])
    return np.array(trainingVideo), np.array(trainingLabels), np.array(testVideo),  np.array(testLabels)

def SplitData(data, rate=0.8):
    cuttendDataCount = int(data.shape[0] * rate)
    group1 = [] 
    group2 = []
    for i in range(len(data)):
        if i < cuttendDataCount:
            group2.append(data[i])
        else:
            group1.append(data[i])
            
    return np.array(group1), np.array(group2)

#imgs, frameCount = LoadVideo('Pig_Identification_Qualification_Train/train/1.mp4')
#pImg = Image.fromarray(center_crop(imgs[5]), mode='RGB')
#pImg.show()
#trainning, testing = SplitData(imgs)
#print(trainning[0].shape)
#
#pImg.show()
#pImg = Image.fromarray(testing[0], mode='RGB')
#pImg.show()

In [4]:
#pImg = Image.fromarray(testing[0], mode='RGB')
#pImg.show()
#wd = np.array([0, 2, 3])
#np.concatenate((wd, wd))

#c = np.concatenate(([None],  [240, 240, 3]))
#test = tf.placeholder(dtype=tf.float32, shape=c, name="test")
#shape = test.get_shape().as_list()
#print(tf.shape(test).as_list())
#print(shape)
#num_features = np.prod(shape[1:])
#print(num_features)

In [5]:
def Spatial_Transfrom (inMap, theta, out_dims=None, **args):
    sess=tf.Session()
    print("inmap{},  shape{}, shapeList{}".format(inMap, tf.shape(inMap), inMap.get_shape().as_list()))
    shape = inMap.get_shape()
    B = tf.shape(inMap)[0]
    H = shape.as_list()[1]
    W = shape.as_list()[2]
    C = shape.as_list()[3]
    
    print('b:{}'.format(B))
    #print('w:{}, shape:{}'.format(w, w.get_shape().as_list()))
    
    #construct theta matrix
    theta = tf.reshape(theta, [B, 2, 3])
    
    if out_dims:
        out_H = out_dims[0]
        out_W = out_dims[1]
        batch_grids = affine_grid_gen(out_H, out_W, theta)
    else:
        batch_grids = affine_grid_gen(H, W, theta)
    
    print('batch_grids')
    print(batch_grids.get_shape().as_list())
    xs = batch_grids[:, 0, :, :]
    ys = batch_grids[:, 1, :, :]
    
    out_samp = bilinear_sampler(inMap, xs, ys)
    print('Spatial_Transfrom')
    print(inMap.get_shape().as_list())
    print(out_samp.get_shape().as_list())
    print('\n')
    return out_samp

def affine_grid_gen(height, width, theta):
    print('affine_grid_gen, height:{}, width:{}, theta:{}'.format(height, width, theta.get_shape().as_list()))
    
    num_batch = tf.shape(theta)[0]
    
    #nornalized grid elements
    x = tf.linspace(-1.0, 1.0, width)
    y = tf.linspace(-1.0, 1.0, height)
    x_t, y_t = tf.meshgrid(x, y)
    print('affine_grid_gen, x:{}, y:{}, x_t:{}, y_t:{}'.format(x.get_shape().as_list(), y.get_shape().as_list(), 
                                                               x_t.get_shape().as_list(), y_t.get_shape().as_list()))
    
    #flatten
    x_t_flat = tf.reshape(x_t, [-1])
    y_t_flat = tf.reshape(y_t, [-1])
    print('affine_grid_gen, x_t_flat:{}, y_t_flat:{}'.format(x_t_flat.get_shape().as_list(), y_t_flat.get_shape().as_list()))
    
    #generate grid
    ones = tf.ones_like(x_t_flat)
    print('ones')
    print(ones.get_shape().as_list())
    sampling_grid = tf.stack([x_t_flat, y_t_flat, ones])
    sampling_grid = tf.expand_dims(sampling_grid, axis=0)
    sampling_grid = tf.tile(sampling_grid, tf.stack([num_batch, 1, 1]))
    
    #cast to float32
    theta = tf.cast(theta, 'float32')
    sampling_grid = tf.cast(sampling_grid, 'float32')
    
    batch_grids = tf.matmul(theta, sampling_grid)
    
    batch_grids = tf.reshape(batch_grids, [num_batch, 2, height, width])
    
    return batch_grids

def bilinear_sampler(img, x, y):
    print("bilinear_sampler x:{}, y:{}".format(str(x), str(y)))
    
    B = tf.shape(img)[0]
    H = tf.shape(img)[1]
    W = tf.shape(img)[2]
    C = tf.shape(img)[3]
    
    max_y = tf.cast(H -1, 'int32')
    max_x = tf.cast(W - 1, 'int32')
    zero = tf.zeros([], dtype='int32')
    
    x = tf.cast(x, 'float32')
    y = tf.cast(y, 'float32')
    
    x = 0.5 * ((x + 1.0) * tf.cast(W, 'float32'))
    y = 0.5 * ((y + 1.0) * tf.cast(H, 'float32'))
    
    x0 = tf.cast(tf.floor(x), 'int32')
    x1 = x0 + 1
    y0 = tf.cast(tf.floor(y), 'int32')
    y1 = y0 + 1
    
    x0 = tf.clip_by_value(x0, zero, max_x)
    x1 = tf.clip_by_value(x1, zero, max_x)
    y0 = tf.clip_by_value(y0, zero, max_y)
    y1 = tf.clip_by_value(y1, zero, max_y)
    
    Ia = get_pixel_value(img, x0, y0)
    Ib = get_pixel_value(img, x0, y1)
    Ic = get_pixel_value(img, x1, y0)
    Id = get_pixel_value(img, x1, y1)
    
    x0 = tf.cast(x0, 'float32')
    x1 = tf.cast(x1, 'float32')
    y0 = tf.cast(y0, 'float32')
    y1 = tf.cast(y1, 'float32')

    wa = (x1-x) * (y1-y)
    wb = (x1-x) * (y-y0)
    wc = (x-x0) * (y1-y)
    wd = (x-x0) * (y-y0)
    
    wa = tf.expand_dims(wa, axis=3)
    wb = tf.expand_dims(wb, axis=3)
    wc = tf.expand_dims(wc, axis=3)
    wd = tf.expand_dims(wd, axis=3)
    
    out = tf.add_n([wa*Ia, wb*Ib, wc*Ic, wd*Id])
    return out

def get_pixel_value(img, x, y):
    """
    Utility function to get pixel value for coordinate
    vectors x and y from a  4D tensor image.
    Input
    -----
    - img: tensor of shape (B, H, W, C)
    - x: flattened tensor of shape (B*H*W, )
    - y: flattened tensor of shape (B*H*W, )
    Returns
    -------
    - output: tensor of shape (B, H, W, C)
    """
    shape = tf.shape(x)
    batch_size = shape[0]
    height = shape[1]
    width = shape[2]

    batch_idx = tf.range(0, batch_size)
    batch_idx = tf.reshape(batch_idx, (batch_size, 1, 1))
    b = tf.tile(batch_idx, (1, height, width))

    indices = tf.stack([b, y, x], 3)

    return tf.gather_nd(img, indices)

In [6]:
#Define Hyper-paramenters
#display_step = 1
learning_rate = 0.001
batch_size = 36
epochs = 30
#num_epochs = 50
num_residual_blocks =22
num_residual_blocks_loc = 5
train_ema_decay = 0.95
require_improvement = 1000
keep_probability = 0.8
keep_probability_loc1 = 0.8
keep_probability_loc2 = 0.8

best_validation_accuracy = 0.0

H, W, C = 64, 64, 3
image_shape = (H, W, C)
image_flat_shape = H * W * C
num_classes = 30

In [7]:
x = tf.placeholder(tf.float32, [None, H, W, C], name='x')
y = tf.placeholder(tf.int32, [None, num_classes], name='y')
#keep_prob_loc1 = tf.placeholder(dtype=tf.float32, name='keep_prob_loc1')
#keep_prob_loc2 = tf.placeholder(dtype=tf.float32, name='keep_prob_loc2')
keep_prob = tf.placeholder(dtype=tf.float32, name='keep_prob')
#vali_x = tf.placeholder(tf.float32, [None, H, W, C])
#vali_y = tf.placeholder(tf.int32, [None, num_classes])
#lr_placeholder = tf.placeholder(dtype=tf.float32, shape=[])
#phase = tf.placeholder(tf.bool, name='phase')

In [8]:
#X_train =  np.array([])
#xx = np.ones((64, 64, 3))
#np.concatenate(X_train, xx)
#LoadVideo('./../Pig_Identification_Qualification_Train/convert/{}.avi'.format(1))

In [9]:
def Label_OneHot(count, y_val):
    y = []
    for i in range(count):
        y.append(np.eye(30, dtype='int32')[y_val])
    return y

#imgs, frameCount = LoadVideo('Pig_Identification_Qualification_Train/convert/{}.avi'.format(i))
def LoadData(show_grid=False, sample_rate = 1):
    imgSet = []
    lablesSet = []
    for i in range(30):
        imgs, frameCount = LoadVideo('./../Pig_Identification_Qualification_Train/convert/{}.avi'.format(i))
        rand.shuffle(imgs)
        lables = Label_OneHot(len(imgs), i)
        for j in range(int(len(imgs) * sample_rate)):
            imgSet.append(imgs[j])
            lablesSet.append(lables[j])
    
    X_train, Y_train, X_test, Y_test = CreateDataSet(imgSet, lablesSet)
    
    if show_grid:
        mask = np.arange(250)
        sample = np.reshape(X_train, [-1, H, W, C])[mask]
        view_images(sample)
        
    X_train = np.reshape(X_train, [-1, H, W, C])
    X_test = np.reshape(X_test, [-1, H, W, C])
    
    X_test, X_val = SplitData(X_test)
    Y_test, Y_val = SplitData(Y_test)

    return X_train, Y_train, X_test, Y_test, X_val, Y_val

def normalize(x):
    """
    Normalize a list of sample image data in the range of 0 to 1
    : x: List of image data.  The image shape is (32, 32, 3)
    : return: Numpy array of normalize data
    """
    # TODO: Implement Function
    a = 0
    b = 255
    return (x-a)/(b-a)

def one_hot_encode(x):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : x: List of sample Labels
    : return: Numpy array of one-hot encoded labels
    """
    # TODO: Implement Function
    num = len(x)
    arr = np.zeros((num, 10))
    for i, xl in enumerate(x):
        arr[i][xl] = 1
    return arr

def SelectSetFromBatchSet(Batch_size, Data1, Data2):
    ids = np.arange(Data1.shape[0])
    rand.shuffle(ids)
    rand.shuffle(ids)
    X_ = []
    Y_ = []
    for i in range(batch_size):
        X_.append(Data1[ids[i]])
        Y_.append(Data2[ids[i]])
    return X_, Y_

def print_stats(session, feature_batch, label_batch, cost, accuracy):
    """
    Print information about loss and validation accuracy
    : session: Current TensorFlow session
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    : cost: TensorFlow cost function
    : accuracy: TensorFlow accuracy function
    """
    # TODO: Implement Function
    loss = session.run(cost, feed_dict={x:feature_batch, y:label_batch, keep_prob:1.0, keep_prob_loc:1.0})
    valid_acc = sess.run(accuracy, feed_dict={
                x: valid_features,
                y: valid_labels,
                keep_prob: 1.,
                keep_prob_loc1:1.,
                keep_prob_loc2:1.
    })
    print('Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                loss,
                valid_acc))
    
def print_stats2(session, feature_batch, label_batch, val_x, val_y, cost, accuracy):
    """
    Print information about loss and validation accuracy
    : session: Current TensorFlow session
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    : cost: TensorFlow cost function
    : accuracy: TensorFlow accuracy function
    """
    # TODO: Implement Function
    miner_val_x, miner_val_y = SelectSetFromBatchSet(feature_batch.shape[0] * 10, val_x, val_y)
    loss = session.run(cost, feed_dict={x:miner_val_x, y:miner_val_y, keep_prob:1.0})
    valid_acc = sess.run(accuracy, feed_dict={
                    x: miner_val_x,
                    y: miner_val_y,
                    keep_prob: 1.,
    })
    
    print('Validation Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                    loss,
                    valid_acc))



def load_Mist_data(root_dir, view_grid=False):
    mnist_cluttered = "mnist_sequence1_sample_5distortions5x5.npz"
    data = np.load(mnist_cluttered)
    X_train, y_train = data['x_train'], data['y_train']
    X_valid, y_valid = data['x_valid'], data['y_valid']
    X_test, y_test = data['x_test'], data['y_test']
    
    if view_grid:
        mask = np.arange(250)
        sample = np.reshape(X_train, [-1, H, W])[mask]
        view_images(sample)
    
    X_train = np.reshape(X_train, [-1, H, W, C])
    X_test = np.reshape(X_test, [-1, H, W, C])
    X_valid = np.reshape(X_valid, [-1, H, W, C])

def random_batch(X, Y, batch_size=batch_size):
    num_train = len(X)
    batch_mask = np.random.choice(num_train, batch_size)
    X_batch = X[batch_mask]
    y_batch = Y[batch_mask]
    
    return X_batch, y_batch

def validate_acc_loss(sess, loss, accuracy, x_val, y_val):
    avg_loss = 0.0
    avg_acc = 0.0
    total_B, batch_idx = generate_batch_indices(x_val)
    #print("x_val shape:{}".format(x_val.shape))
    for i in range(total_B):
        #create batch
        idx = batch_idx[i]
        mask = np.arange(idx[0], idx[1])
        #print("validate_acc_loss mask:{}".format(mask))
        batch_xs, batch_ys = x_val[mask], y_val[mask]
        
        val_feed_dict = {X:batch_xs, Y:batch_ys, phase:True}
        l, a = sess.run([loss, accuracy], feed_dict=val_feed_dict)
        
        avg_loss += 1 / total_batch
        avg_acc += a / total_batch
    
    return avg_loss, avg_acc

def test_acc(sess, accuracy, X_test, y_test):
    avg_acc = 0.0
    total_B, batch_idx = generate_batch_indices(X_test)
    
    for i in range(total_B):
        idx = batch_idx[i]
        mask = np.arange(inx[0], idx[1])
        batch_xs, batch_ys = X_test[mask], y_test[mask]
        
        test_feed_dict = {X:batch_xs, y:batch_ys, phase: Fase}
        a = sess.run(accuracy, feed_dict=test_feed_dict)
        
        avg_acc += a / total_B
    
    return avg_acc

def generate_batch_indices(X, batch_size=batch_size):
    num_train = len(X)
    total_B = int(np.ceil(num_train / float(batch_size)))
    batch_indices = [(i * batch_size, min(num_train, (i+1) * batch_size)) for i in range(0, total_B)]
    return total_B, batch_indices


def train_neural_network(session, optimizer, feature_batch, label_batch):
    """
    Optimize the session on a batch of images and labels
    : session: Current TensorFlow session
    : optimizer: TensorFlow optimizer function
    : keep_probability: keep probability
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    """
    # TODO: Implement Function
    
    session.run(optimizer, feed_dict={x:feature_batch, y:label_batch,  phase:True})
#a, b, c, d = LoadData(True)

In [11]:
#preprocess_and_data(LoadData, normalize)

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def non_one_hot(x):
    return x

def one_hot_decode(y):
    x = y[:][:]

# Preprocess Training, Validation, and Testing Data
#helper.preprocess_and_save_data(cifar10_dataset_folder_path, normalize, one_hot_encode)
#valid_features, valid_labels = pickle.load(open('preprocess_validation.p', mode='rb'))

In [13]:
#im = np.arange(311040).reshape((240, 432, 3))
#center_crop(im)
#mnist_cluttered = "./mnist_sequence1_sample_5distortions5x5.npz"
#data = np.load('./mnist_sequence1_sample_5distortion5x5.npz')
#X_train = data['X_valid']
#pImg = Image.fromarray(X_train)
#pImg.show()

In [14]:
def loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    print(logits.get_shape().as_list())
    print(labels.get_shape().as_list())
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    return cross_entropy_mean

def top_k_error(predictions, labels, k):
    batch_size = predictions.get_shape().as_list()[0]
    top = tf.nn.in_top_k(predictions, labels, k=1)
    #in_top1 = tf.to_float(top)
    num_correct = tf.reduce_sum(top)
    return (batch_size - num_correct) / float(batch_size)

def train_operation(global_step, total_loss):
    #tf.summary.scalar('learning_rate', lr_placeholder)
    #tf.summary.scalar('train_loss', total_loss)
    #tf.summary.scalar('train_top1_error', top1_error)
    
    #ema = tf.train.ExponentialMovingAverage(train_ema_decay, global_step)
    #train_ema_op = ema.apply([total_loss, top1_error])
    #tf.summary.scalar('train_top1_error_avg', ema.average(top1_error))
    #tf.summary.scalar('train_loss_avg', ema.average(total_loss))
    
    opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = opt.minimize(total_loss, global_step=global_step)
    return train_op

def validation_op(validation_step, top1_error, loss):
    ema = tf.train.ExponentialMovingAverage(0.0, validation_step)
    ema2 = tf.train.ExponentialMovingAverage(0.95, validation_step)
    val_op = tf.group(validation_step.assign_add(1), ema.apply([top1_error, loss]),
                          ema2.apply([top1_error, loss]))
    top1_error_val = ema.average(top1_error)
    top1_error_avg = ema2.average(top1_error)
    loss_val = ema.average(loss)
    loss_val_avg = ema2.average(loss)
    
    tf.summary.scalar('val_top1_error', top1_error_val)
    tf.summary.scalar('val_top1_error_avg', top1_error_avg)
    tf.summary.scalar('val_loss', loss_val)
    tf.summary.scalar('val_loss_avg', loss_val_avg)
    return val_op


def BatchingData(batch_size, X_train, Y_train, trainBatchCount):
    """
    Process batching data
    """
    for i in range(trainBatchCount):
        start_t = i*batch_size
        X_t_batch = np.cast['float'](X_train[start_t:start_t+batch_size])
        Y_t_batch = np.cast['float'](Y_train[start_t:start_t+batch_size])
        
        yield X_t_batch, Y_t_batch, i

        
def interpolate_bilinear(src, ri, rf, rc, ti, tf, tc):  
    if rf == rc & tc == tf:  
        out = src[rc][tc]  
    elif rf == rc:  
        out = (ti - tf)*src[rf][tc] + (tc - ti)*src[rf][tf]  
    elif tf == tc:  
        out = (ri - rf)*src[rc][tf] + (rc - ri)*src[rf][tf]  
    else:  
        inter_r1 = (ti - tf)*src[rf][tc] + (tc - ti)*src[rf][tf]  
        inter_r2 = (ti - tf)*src[rc][tc] + (tc - ti)*src[rc][tf]  
        out = (ri - rf)*inter_r2 + (rc - ri)*inter_r1  
  
    return out

def load_preprocess_batch(path, batch_size, dataCount):
    features, labels = pickle.load(open(path, mode='rb'))
    trainBatchCount = int(features.shape[0] / batch_size)
    dataCount[0] = trainBatchCount
    
    return BatchingData(batch_size, features, labels, trainBatchCount)

In [23]:
def file_name(file_dir):
    L=[]
    num=[]
    for root, dirs, files in os.walk(file_dir):
        #print(files)
        for file in files:
            if os.path.splitext(file)[1] == '.JPG':
                p = os.path.join(root, file)
                n = os.path.splitext(file)[0]
                yield p, n
    #return L, num

def loadImgsp():
    _x_ = []
    paths = []
    for path, num in file_name('../Pig_Identification_Qualification_Train/test_set'):
        _x = cv2.imread(path)[:,:,::-1]
        pImg = Image.fromarray(_x, mode='RGB')
        pImg = pImg.resize((64,64))
        _x_.append(np.array(pImg))
        paths.append(num)
        #_y_.append(Label_OneHot(1, rand.randint(1, 30)))
    return {'data':np.array(_x_), 'path':paths}

def loadImgsAndSave():
    _x_ = []
    paths = []
    for path, num in file_name('../Pig_Identification_Qualification_Train/test_set'):
        _x = cv2.imread(path)[:,:,::-1]
        pImg = Image.fromarray(_x, mode='RGB')
        if _x.shape[0] > _x.shape[1]:
            pImg.transpose(Image.ROTATE_90)
        pImg = pImg.resize((64,64))
        _x_.append(np.array(pImg))
        paths.append(num)
        #_y_.append(Label_OneHot(1, rand.randint(1, 30)))
    result = {'data':np.array(_x_), 'path':paths}
    f = open('test_set.p', 'wb')
    pickle.dump((result), f)
    f.close()
    
def loadImgs():
    f = open('test_set.p', mode='rb')
    result = pickle.load(f)
    f.close()
    return result

#path, num = file_name('../Pig_Identification_Qualification_Train/test_set')
#print(num)
#loadImgsAndSave()
xx = loadImgs()
print(xx['data'].shape)

(3000, 64, 64, 3)


In [16]:
from sklearn.preprocessing import LabelBinarizer
ww = rand.randint(1, 30)
print(ww)
yy = np.array(Label_OneHot(1, ww))
print(yy.shape)
label_binarizer = LabelBinarizer()
label_binarizer.fit(range(30))
label_ids = label_binarizer.inverse_transform(yy)
print(label_ids)

1
(1, 30)
[1]


In [17]:
#X_train, Y_train, X_test, Y_test, X_val, Y_val = LoadData(True, sample_rate = 0.5)

global_step = tf.Variable(0, trainable=False)
validation_step = tf.Variable(0, trainable=False)

#num_train = X_train.shape[0]
#num_val = X_val.shape[0]

#logit_loc2 = inference(x, num_residual_blocks_loc, dropout=0.92, reuse=False, num_class=6)
#h_trans2 = Spatial_Transfrom(x, logit_loc2)
logits = inference(x, num_residual_blocks, dropout=keep_prob, reuse=False, num_class=num_classes)
logits = tf.identity(logits, name='logits')
#vali_logits = inference(vali_x, num_residual_blocks,   reuse=True)

regu_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
train_loss = loss(logits, y)

full_loss = tf.add_n([train_loss] + regu_losses)

#predictions = tf.nn.softmax(logits)

correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
#print(predictions.get_shape().as_list())
#train_top1_error = top_k_error(predictions, vali_y, 1)

#vali_loss = loss(vali_logits, vali_y)
#vali_predictions = tf.nn.softmax(vali_logits)
#vali_top1_error = top_k_error(vali_predictions, vali_y, 1)

train_op = train_operation(global_step, full_loss)
#val_op  = validation_op(validation_step, vali_top1_error, vali_loss)

saver = tf.train.Saver(tf.global_variables())
summary_op = tf.summary.merge_all()
#summary_writer = tf.train.SummaryWriter('/notran_small_logs', sess.graph)
#init = tf.initialize_all_variables()

X_val, Y_val = pickle.load(open('./preprocess_batch_val.p', mode='rb'))

[None, 30]
[None, 30]


In [ ]:
def saveOutput(sess, number):    
    path__ = []
    ww = rand.randint(1, 25)
    batch_count = int(xx['data'].shape[0]/batch_size)
    text_=''
    __text = ''
    _text_ = ''
    for _x_, _y_, batch_i in BatchingData(batch_size, xx['data'], np.array(Label_OneHot(3000, ww)), batch_count):
        __x = normalize(_x_)
        predictions  = sess.run(logits, feed_dict={x:__x, keep_prob:1.})
        random_test_predictions  = sess.run(tf.nn.top_k(tf.nn.softmax(logits), 30), feed_dict={x:__x,  keep_prob:1.})
        
        label_binarizer = LabelBinarizer()
        label_binarizer.fit(range(30))
        label_ids = label_binarizer.inverse_transform(predictions)
        
        print('label_ids shape{},test_predictions shape {}'.format(label_ids.shape, random_test_predictions.values.shape))
        
        for idx in range(batch_size):
            _text_ += '{}, {}, {}, {}\n'.format(xx['path'][idx + (batch_i * batch_size)], 
                                           random_test_predictions.indices[idx][0], random_test_predictions.values[idx][0], label_ids[idx])
            
            for idxx in range(30):
                __text += '{}, {}, {:.15f}\n'.format(xx['path'][idx + (batch_i * batch_size)], 
                                           random_test_predictions.indices[idx][idxx]+1, random_test_predictions.values[idx][idxx])
            
            if idx % 5 == 0:
                text_ += 'filename : {}, Label: {}, Prediction: {}\n'.format(xx['path'][idx + (batch_i * batch_size)], 
                                           label_ids[idx], random_test_predictions.values[idx][:3])
                #print(text_)
            #text_ += '\n'

        #print(text_)
        
       
        path__ = []
    
    #print(__text[0:200])
    f2 = open('./output3_{}.csv'.format(number), 'w')
    f2.write(__text)
    f2.close()
    f1=open('./output3_log_{}.log'.format(number), 'w')
    f1.write(text_)
    f1.close()
    f3 = open('./output3_d_{}.csv'.format(number), 'w')
    f3.write(_text_)
    f3.close()

with tf.Session() as sess:
    # Initializing the variables
    
    sess.run(tf.global_variables_initializer())
    
    writer = tf.summary.FileWriter("./tran_small3",sess.graph)  
   # trainBatchCount = int(X_train.shape[0] / batch_size)
   # valBatchCount = int(X_val.shape[0] / batch_size)
  #  print("Training on {} samples, validating on{}".format(len(X_train), len(X_val)))
    #fig = plot.figure()
    #view_images(X_train[0])
    # Training cycle
    
    for i in range(epochs):
        epoch = i
        start_time = time.time()
        batch_i = 0
        dataCount = [0]
        for x_t, y_t, batch_i in load_preprocess_batch('preprocess_batch_tranning.p', batch_size, dataCount):
            if batch_i == epoch:
                view_images(x_t)
                print(y_t)
                
            _, train_loss_value, = sess.run([train_op, full_loss], {x:x_t, y:y_t, 
                                                                    keep_prob:keep_probability})
            #thetas = sess.run(h_trans2, feed_dict={x:x_t, keep_prob_loc1:1, keep_prob_loc2:1})
            if batch_i % int(dataCount[0] / 100) == 0:
                print('Trainning batch {}/{} in epoch {}, local loss is {:.4f}'.format(batch_i, dataCount[0], epoch+1, train_loss_value))
            
            if batch_i >= dataCount[0]:
                summary_str = sess.run(summary_op, feed_dict={x:x_t, y:y_t, 
                                                                    keep_prob:keep_probability})
                writer.add_summary(summary_str, batch_i)
            
            
        #thetas = sess.run(h_trans, feed_dict={x:batch_features, phase:True})
        #thetas = thetas[0:9].squeeze()
        #thetas = sess.run(h_trans2, feed_dict={x:x_t,keep_prob:1. })
        #view_images(thetas[0:9])
        #rint ('Train top1 error = '+ train_error_value)
        #print ('Validation top1 error = {:.4f}'.format(validation_error_value))
        #print ('Validation loss = '+validation_loss_value)
        print('Epoch {:>2}'.format(epoch+1))
        
        print_stats2(sess, x_t, y_t, X_val, Y_val, full_loss, accuracy)
        
        try:
            if i == 4:
                saveOutput(sess, 5)

            if i == 9:
                saveOutput(sess, 10)

            if i == 14:
                saveOutput(sess, 15)

            if i == 24:
                saveOutput(sess, 15)
        except Exception as e3:
            
            pass
        
        
        #thetas = thetas[0:9].squeeze()
        #plot.clf()
        #view_images(thetas[0:9])
        #fig.canvas.draw()
        #plot.savefig('/convert240/plotting_figure/epoch_'+str(epoch)+'_thetas.png', bbox_inches='tight')
        
        duration = time.time() - start_time
        examples_per_sec = batch_size / duration
        
        format_str = ('epoch Summary: {}: loss = {:.4f} ({:.1f} examples/sec; {:.3f} ' 'sec/batch)')
        print (format_str.format(datetime.now(), train_loss_value, examples_per_sec, duration))
        print ('----------------------------')
        
        #plot.clf()
        #print('Print theta')
        #for j in range(9):
        #    plot.subplot(3,3, j+1)
        #    plot.imshow(thetas[j], cmap='gray')
        #    plot.axis('off')
        
    save_model_path = './Pigs_identity_Tran_small3'
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)
    
    test_total_acc = 0
    
    for feature, labels, batch_i in  load_preprocess_batch('./preprocess_batch_testing.p', batch_size, dataCount):
        test_total_acc += sess.run(accuracy, feed_dict={x:feature, y:labels, keep_prob:1.})
        print("the batch at {}, local acc is {}".format(batch_i, test_total_acc))
        
    print("Testing Accuracy: {}\n".format(test_total_acc/batch_i))
    saveOutput(sess, 'end')
    


In [ ]:

#print( rand.randint(1, 30))
#print(Label_OneHot(30, rand.randint(1, 30)))
#cap = cv2.VideoCapture('../Pig_Identification_Qualification_Train/test_set/10.jpg')
tempx = cv2.imread('../Pig_Identification_Qualification_Train/test_set/10.jpg', 1)[:,:,::-1]
pImg = Image.fromarray(tempx, mode='RGB')
pImg = pImg.resize((64,64))
oo2 = np.array([np.array(pImg)])
tempx = np.array([tempx])
print(oo2.shape)
print(tempx.shape)
#print(tempx[[:,:,::-1]])
view_images(oo2)
view_images(tempx)



In [20]:
#X_train, Y_train, X_test, Y_test, X_val, Y_val = LoadData(True, sample_rate = 0.2)


save_model_path = './Pigs_identity_Tran_small3'
n_samples = 4
top_n_predictions = 3
loaded_graph = tf.get_default_graph()

with tf.Session() as sess:
    loader = tf.train.import_meta_graph(save_model_path+'.meta')
    loader.restore(sess, save_model_path)
    
    print(tf.GraphKeys)
    
    loaded_x = loaded_graph.get_tensor_by_name('x:0')
    loaded_y = loaded_graph.get_tensor_by_name('y:0')
    #loaded_keep_prob_loc1 = loaded_graph.get_tensor_by_name('keep_prob_loc1:0')
    #loaded_keep_prob_loc2 = loaded_graph.get_tensor_by_name('keep_prob_loc2:0')
    loaded_keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    loaded_logits = loaded_graph.get_tensor_by_name('logits:0')
    loaded_acc = loaded_graph.get_tensor_by_name('accuracy:0')
    
    print('loaded_x{} shape:{}'.format(loaded_x, loaded_x.get_shape()))
    
    #a = Label_OneHot(30, rand.randint(1, 30))
    #b = Label_OneHot(30, rand.randint(1, 30))
    #tempx = np.random.rand(2, 64, 64, 3)
    #tempy = np.array(Label_OneHot(2, rand.randint(1, 30)))
    
    #test_total_acc = sess.run(loaded_acc, feed_dict={loaded_x:tempx, loaded_y:tempy, 
                                                        #  loaded_keep_prob_loc1:1., loaded_keep_prob_loc2:1., loaded_keep_prob:1.})
    #print('Test Run:{}'.format(test_total_acc))
    test_total_acc = 0
    #testBatchCount = int(X_test.shape[0] / batch_size)
    datacount = [0]
    #for feature, labels, batch_i in  load_preprocess_batch('./preprocess_batch_testing.p', batch_size, datacount):
        #print(feature)
        #print(batch_i)
        #test_total_acc += sess.run(loaded_acc, feed_dict={loaded_x:feature, loaded_y:labels, loaded_keep_prob:1.})
        #print("the batch at {}, local acc is {}".format(batch_i, test_total_acc))
        
    #print("Testing Accuracy: {}\n".format(test_total_acc/datacount[0]))
    
    saveOutput(sess, 'end')

INFO:tensorflow:Restoring parameters from ./Pigs_identity_Tran_small3
<class 'tensorflow.python.framework.ops.GraphKeys'>
loaded_xTensor("x:0", shape=(?, 64, 64, 3), dtype=float32) shape:(?, 64, 64, 3)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
label_ids shape(36,),test_predictions shape (36, 30)
lab

In [ ]:
print(__text[0:200])
f2 = open('./output1.csv', 'w')
f2.write(__text)
f2.close()
f1=open('./output_log.log', 'w')
f1.write(text_)
f1.close()

In [22]:
import csv

def logloss (predictions):
    totalLoss = 0
    logPre = np.log(predictions)
    for n in range(logPre.shape[0]):
        totalLoss += logPre[n]
    logLoss = - 1 / len(predictions) * totalLoss
    print(logLoss)
    
nump = []
with open('./output3_d_end.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        nump.append(float(row[2]))
nump = np.array(nump)
print(nump.shape)
logloss(nump)



(2988,)
0.344289085669


In [ ]:
nump = []
with open('./output1.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        nump.append(row)

nump = np.cast['float'](np.array(nump))
kkk = []
for jjj in range(int(nump.shape[0] / 30)):
    lll = nump[jjj * 30:jjj * 30 + 30]
    #print(lll)
    lll.sort(axis=0)
    for eles in range(lll.shape[0]):
        kkk.append(lll[eles])
kkk = np.array(kkk)
print(kkk[:35])


#nump.sort(axis=0, )
#print(nump[:35])

In [ ]:
print(kkk.shape)
with open('./output1_new.csv', 'w') as w:
    writer = csv.writer(w)
    for jjs in range(kkk.shape[0]):
        writer.writerow([str(int(kkk[jjs][0])), str(int(kkk[jjs][1])), '{:.15f}'.format(kkk[jjs][2])])

In [ ]:
X_train, Y_train, X_test, Y_test, X_val, Y_val = LoadData(True, sample_rate = 0.1)

global_step = tf.Variable(0, trainable=False)
validation_step = tf.Variable(0, trainable=False)

num_train = X_train.shape[0]
num_val = X_val.shape[0]

logit_loc1 = inference(x, num_residual_blocks_loc, dropout=keep_prob_loc1, reuse=False, num_class=6)
h_trans1 = Spatial_Transfrom(x, logit_loc1)
logit_loc2 = inference(h_trans1, num_residual_blocks_loc, dropout=keep_prob_loc2, reuse=False, num_class=6)
h_trans2 = Spatial_Transfrom(h_trans1, logit_loc1, out_dims=[H, H])
logits = inference(h_trans2, num_residual_blocks, dropout=keep_prob, reuse=False, num_class=num_classes)
logits = tf.identity(logits, name='logits')
#vali_logits = inference(vali_x, num_residual_blocks,   reuse=True)

regu_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
train_loss = loss(logits, y)

full_loss = tf.add_n([train_loss] + regu_losses)

#predictions = tf.nn.softmax(logits)

correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
#print(predictions.get_shape().as_list())
#train_top1_error = top_k_error(predictions, vali_y, 1)

#vali_loss = loss(vali_logits, vali_y)
#vali_predictions = tf.nn.softmax(vali_logits)
#vali_top1_error = top_k_error(vali_predictions, vali_y, 1)

train_op = train_operation(global_step, full_loss)
#val_op  = validation_op(validation_step, vali_top1_error, vali_loss)

saver = tf.train.Saver(tf.global_variables())
summary_op = tf.summary.merge_all()
#init = tf.initialize_all_variables()

with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    trainBatchCount = int(X_train.shape[0] / batch_size)
    valBatchCount = int(X_val.shape[0] / batch_size)
    print("Training on {} samples, validating on{}".format(len(X_train), len(X_val)))
    fig = plot.figure()
    view_images(X_train[0])
    # Training cycle
    
    for i in range(epochs):
        epoch = i
        start_time = time.time()
        batch_i = 0
        for x_t, y_t, batch_i in  BatchingData(batch_size, X_train, Y_train, trainBatchCount):
            _, train_loss_value, = sess.run([train_op, full_loss], {x:x_t, y:y_t, 
                                                                    keep_prob:keep_probability, 
                                                                    keep_prob_loc1:keep_probability_loc1,
                                                                    keep_prob_loc2:keep_probability_loc2})
            thetas = sess.run(h_trans2, feed_dict={x:x_t, keep_prob_loc1:1, keep_prob_loc2:1})
            
        #thetas = sess.run(h_trans, feed_dict={x:batch_features, phase:True})
        #thetas = thetas[0:9].squeeze()
        
        #rint ('Train top1 error = '+ train_error_value)
        #print ('Validation top1 error = {:.4f}'.format(validation_error_value))
        #print ('Validation loss = '+validation_loss_value)
        print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch, batch_i), end='')
        print_stats2(sess, x_t, y_t, X_val, Y_val, full_loss, accuracy)
        
        #thetas = thetas[0:9].squeeze()
        plot.clf()
        view_images(thetas)
        for j in range(9):
            plot.subplot(3,3, j+1)
            plot.imshow(thetas[j], cmap='gray')
            plot.axis('off')
        fig.canvas.draw()
        plot.savefig('./plotting_figure/epoch_'+str(epoch)+'_thetas.png', bbox_inches='tight')
        
        duration = time.time() - start_time
        examples_per_sec = batch_size / duration
        
        format_str = ('epoch Summary: {}: loss = {:.4f} ({:.1f} examples/sec; {:.3f} ' 'sec/batch)')
        print (format_str.format(datetime.now(), train_loss_value, examples_per_sec, duration))
        print ('----------------------------')
        
        #plot.clf()
        #print('Print theta')
        #for j in range(9):
        #    plot.subplot(3,3, j+1)
        #    plot.imshow(thetas[j], cmap='gray')
        #    plot.axis('off')
        
        
        

In [10]:
"""
def init_weights(name, shape):
    init = tf.contrib.layers.variance_scaling_initializer()
    w = tf.get_variable(name, shape, tf.float32, init)
    
    return w

def init_bias(name, shape, trans=False):
    init = tf.zeros_initializer
    b = tf.get_variable(name, shape, tf.float32, init)
    if trans:
        x = np.array([[1., 0, 0], [0, 1., 0]])
        x = x.astype('float32').flatten()
        b = tf.Variable(initial_value=x)
    return b

def Conv2D(input_tensor, input_shape, filter_size, num_filters, strides=1, name=None):
    
    #conv net helper
    
    shape = [filter_size, filter_size, input_shape, num_filters]
    w = init_weights(name=name+'_w', shape=shape)
    b = init_bias(name=name+'_b', shape=shape[-1])
    
    conv = tf.nn.conv2d(input_tensor, w, strides=[1, strides, strides, 1], padding='SAME', name=name)
    conv = tf.nn.bias_add(conv, b)
    return conv
    
def MaxPolling2D(input_tensor, k=2, use_relu=False, name=None):
    pool = tf.nn.max_pool(input_tensor, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME', name=name)
    
    if use_relu:
        pool = tf.nn.relu(pool)
    
    return pool

def Flatten(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:].num_elements()
    layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat, number_features

def BatchNormalization(input_tensor, phase, use_relu=False, name=None):
    normed = tf.contrib.layers.batch_norm(input_tensor, center=True, is_trainning=phase, scope=name)
    if use_relu:
        normed = tf.nn.relu(normed)
    return normed

def Dense(input_tensor, num_inputs, num_outputs, use_relu=True, trans=False, name=None):
    shape = [num_inputs, num_outputs]
    
    w = init_weights(name=name+'_w', shape=shape)
    b = init_bias(name=name+'_b', shape=shape[-1], trans=trans)
    
    fc = tf.matmul(input_tensor, w, name=name) + b
    
    if use_relu:
        fc = tf.nn.relu(fc)
        
    return fc

def theta_bias(name):
    with tf.variable_scope(name):
        x = np.array([1., 0, 0], [0, 1., 0])
        x = x.astype('float32').flatten()
        return tf.Variable(initial_value=x)
"""
#w = init_weights('testW', image_shape)
#b = init_bias('testB', image_shape)
#convTest = Conv2D(X, 1, 5, 32, name="convTest")
#poolTest = MaxPolling2D(X, use_relu=True, name='poolTest')
#flat = Flatten(X)
#dense = Dense(X, 2048, 512, use_relu=True, trans=True, name='FCTest')

"\ndef init_weights(name, shape):\n    init = tf.contrib.layers.variance_scaling_initializer()\n    w = tf.get_variable(name, shape, tf.float32, init)\n    \n    return w\n\ndef init_bias(name, shape, trans=False):\n    init = tf.zeros_initializer\n    b = tf.get_variable(name, shape, tf.float32, init)\n    if trans:\n        x = np.array([[1., 0, 0], [0, 1., 0]])\n        x = x.astype('float32').flatten()\n        b = tf.Variable(initial_value=x)\n    return b\n\ndef Conv2D(input_tensor, input_shape, filter_size, num_filters, strides=1, name=None):\n    \n    #conv net helper\n    \n    shape = [filter_size, filter_size, input_shape, num_filters]\n    w = init_weights(name=name+'_w', shape=shape)\n    b = init_bias(name=name+'_b', shape=shape[-1])\n    \n    conv = tf.nn.conv2d(input_tensor, w, strides=[1, strides, strides, 1], padding='SAME', name=name)\n    conv = tf.nn.bias_add(conv, b)\n    return conv\n    \ndef MaxPolling2D(input_tensor, k=2, use_relu=False, name=None):\n    poo